In [ ]:
import pandas as pd
import numpy as np
from numpy.random import choice
from random import random, randint, sample
from collections import defaultdict
import scipy.stats

In [ ]:
import array
l = [[1, 2, 3], [1, 2, 3]]
row = [1, 2, 3]
col = [4, 4, 4]

l.append(row)
for row, val in zip(l, col):
    row.append(val)
l

In [ ]:
import multiprocessing
import sys
import collections
from random import random
from time import sleep
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

#import matplotlib.animation as animation

from matplotlib import animation, rc
from IPython.display import HTML

from matplotlib.lines import Line2D

import seaborn as sns
sns.set()
sns.set_style('ticks')
# sns.set_style('dark')

#colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']
colors = sns.color_palette("Set1", 9)
#colors = sns.color_palette("rainbow", 8)

#sns.palplot(colors)

In [ ]:
%matplotlib inline
# Make inline plots vector graphics instead of raster graphics
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina', 'png')
# set_matplotlib_formats('svg', 'pdf')


We can sample drives between 0 and 1 from a beta:

In [ ]:
plt.figure(figsize=(8, 5))
x = np.linspace(0.0001, 0.9999, 100)
with sns.axes_style('whitegrid'):
    for a, b in [(5, 1), (2, 5), (1, 5), (1, 1)]:
        plt.plot(x, scipy.stats.beta.pdf(x, a, b), label='alpha: {}, beta: {}'.format(a, b))
    plt.legend(loc='upper right') ;

... but it does not matter which distribution such allele specivic values are drawn from. When we compute drive as `x_drive / (x_drive + y_drive)` this is always uniform:

In [ ]:
plt.figure(figsize=(8, 5))
with sns.axes_style('white'):
    x = np.linspace(0.0001, 0.9999, 100000)
    for a, b in [(15, 1), (10, 1), (5, 1)]:
        x_drive = abs(np.random.normal(a, b, size=len(x)))
        y_drive = abs(np.random.normal(a, b, size=len(x)))
        plt.hist(x_drive / (x_drive + y_drive), density=True, bins=200, alpha=0.2, label='alpha: {}, beta: {}'.format(a, b))
    plt.legend() ;

In [ ]:
plt.figure(figsize=(8, 5))
with sns.axes_style('white'):
    x = np.linspace(0.0001, 0.9999, 100000)
    for a, b in [(15, 1), (10, 1), (5, 1)]:
        x_drive = np.random.beta(a, b, size=len(x))
        y_drive = np.random.beta(a, b, size=len(x))
        plt.hist(x_drive / (x_drive + y_drive), bins=200, alpha=0.2, label='alpha: {}, beta: {}'.format(a, b))
    plt.legend() ;

In [ ]:
DRIVES = pd.DataFrame()

class Allele:
    __slots__ = ('chrom', 'label', 'val')
    
    def __init__(self, chrom, label, val):
        self.chrom = chrom
        self.label = label
        self.val = val
    
class Male:
    __slots__ = ('x_allele', 'y_allele')
        
    def __init__(self, x_allele, y_allele):
        assert x_allele.chrom == 'X'
        assert y_allele.chrom == 'Y'
        self.x_allele = x_allele
        self.y_allele = y_allele

    def __add__(self, female):
        tot_val = self.x_allele.val + self.y_allele.val

        x_drive = DRIVES[self.x_allele.label][self.y_allele.label]
        y_drive = DRIVES[self.y_allele.label][self.x_allele.label]

        x_proportion = x_drive / (x_drive + y_drive)
                
        if random() < x_proportion:
            sperm_allele = self.x_allele
        else:
            sperm_allele = self.y_allele

        if random() < 0.5:
            egg_allele = female.x1_allele
        else:        
            egg_allele = female.x2_allele
  
        if sperm_allele.chrom == 'Y':
            return Male(egg_allele, sperm_allele)
        else:
            return Female(egg_allele, sperm_allele)
            
class Female:
    __slots__ = ('x1_allele', 'x2_allele')

    def __init__(self, x1_allele, x2_allele):
        assert x1_allele.chrom == 'X'
        assert x2_allele.chrom == 'X'
        self.x1_allele = x1_allele
        self.x2_allele = x2_allele

    def __add__(self, male):
        return male + self

    
class Population:
    __slots__ = ('males', 'females', 'cur_xlabel', 'cur_ylabel', 'alpha', 'beta', 'pop_size', 'drive_coef')
            
    def __init__(self, males=[], females=[], cur_xlabel=0, cur_ylabel=1, alpha=5, beta=1):
        self.males = males
        self.females = females
        self.alpha = alpha
        self.beta = beta
        self.pop_size = len(self.males) + len(self.females)
        self.drive_coef = defaultdict(dict)
        self.cur_xlabel = cur_xlabel # label of latest x allele
        self.cur_ylabel = cur_ylabel # label of latest y allele
        
    def initialize(self, pop_size):
        assert not pop_size % 2, 'initial pop_size must be even'
        self.males = [Male(Allele('X', self.cur_xlabel, 1), 
                           Allele('Y', self.cur_ylabel, 1)) for _ in range(pop_size//2)]
        self.females = [Female(Allele('X', self.cur_xlabel, 1), 
                               Allele('X', self.cur_xlabel, 1)) for _ in range(pop_size//2)]

        self.update_drives(self.cur_xlabel)
        self.update_drives(self.cur_ylabel)

#     def update_drives(self, label):
#         # assert label >= len(self.DRIVES)

#         nrow = len(self.DRIVES[0]) if self.DRIVES else 1
#         row = np.random.normal(self.alpha, self.beta, size=nrow).tolist()
#         self.DRIVES.append(row)
        
#         ncol = len(self.DRIVES)
#         col = np.random.normal(self.alpha, self.beta, size=ncol).tolist()
#         for row, val in zip(self.DRIVES, col):
#             row.append(val)
            
    def update_drives(self, label):  
        global DRIVES

        assert label not in DRIVES.columns.values
        nrows, ncols = DRIVES.shape
#         Population.DRIVES[label] = np.random.beta(self.alpha, self.beta, size=nrows)
        DRIVES[label] = np.random.normal(self.alpha, self.beta, size=nrows)
        nrows, ncols = DRIVES.shape
#         new_row = pd.DataFrame.from_records([np.random.beta(self.alpha, self.beta, size=ncols)], 
#                                             columns=Population.DRIVES.columns)
        new_row = pd.DataFrame.from_records([np.random.normal(self.alpha, self.beta, size=ncols)], 
                                            columns=DRIVES.columns)
        new_row.index = [label]
        DRIVES = pd.concat([DRIVES, new_row])
        DRIVES = DRIVES.copy() 
        
    def resample(self, new_pop_size=None):
        if new_pop_size is None:
            new_pop_size = self.pop_size
        new_males = list()
        new_females = list()
        
        for i in range(new_pop_size):
            male = self.males[int(random() * len(self.males))]
            female = self.females[int(random() * len(self.females))]
            child = male + female
                        
            if isinstance(child, Male):
                new_males.append(child)
            else:
                new_females.append(child)       
                
        return Population(new_males, new_females, self.cur_xlabel, self.cur_ylabel)

    def mutate(self, x_mut_prob, y_mut_prob):
        for i in range(len(self.males)):
            if random() < x_mut_prob:
                self.cur_xlabel += 2
                self.males[i].x_allele = Allele('X', self.cur_xlabel, 1)
                
                self.update_drives(self.cur_xlabel)
                
            if random() < y_mut_prob:
                self.cur_ylabel += 2
                self.males[i].y_allele = Allele('Y', self.cur_ylabel, 1)

                self.update_drives(self.cur_ylabel)

        for i in range(len(population.females)):
            if random() < x_mut_prob:
                self.cur_xlabel += 2
                self.females[i].x1_allele = Allele('X', self.cur_xlabel, 1)
                
                self.update_drives(self.cur_xlabel)
                
            if random() < x_mut_prob:
                self.cur_xlabel += 2
                self.females[i].x2_allele = Allele('X', self.cur_xlabel, 1)
                
                self.update_drives(self.cur_xlabel)
                        
    def x_freqs(self):
        alleles = [male.x_allele.label for male in self.males] + \
            [female.x1_allele.label for female in self.females] + \
            [female.x2_allele.label for female in self.females]
        unique_elements, counts_elements = np.unique(np.array(alleles), return_counts=True)
        return unique_elements, counts_elements / counts_elements.sum()

    def y_freqs(self):
        alleles = [male.y_allele.label for male in self.males]
        unique_elements, counts_elements = np.unique(np.array(alleles), return_counts=True)
        return unique_elements, counts_elements / counts_elements.sum()

    def male_freq(self):
        return len(self.males) / (len(self.males) + len(self.females))  
    

pop_size_history = [10000] * 2000
x_mutation_rate = 0.0001
y_mutation_rate = 0.0001

# alpha = 1
# beta = 5
alpha = 5
beta = 1

_x = np.random.normal(alpha, beta, size=100000)
_y = np.random.normal(alpha, beta, size=100000)
null_dist = _x / (_x + _y)

population = Population(alpha=alpha, beta=beta)
population.initialize(pop_size_history.pop(0))

generations = 1000

x_data = collections.OrderedDict()
y_data = collections.OrderedDict()
nr_lines = 50000
for i in range(nr_lines):
    x_data[i] = []
    y_data[i] = [] 
x_data['nr_x_alleles'] = []
y_data['nr_x_alleles'] = []
x_data['nr_y_alleles'] = []
y_data['nr_y_alleles'] = []
x_data['male_freq'] = []
y_data['male_freq'] = []
x_data['pop_size'] = []
y_data['pop_size'] = []

for gen in range(1, generations+1):
    
    pop_size = pop_size_history.pop(0)
    population = population.resample(pop_size)
    population.mutate(x_mut_prob=x_mutation_rate, y_mut_prob=y_mutation_rate)
    x_alleles, x_freqs = population.x_freqs()
    y_alleles, y_freqs = population.y_freqs()
    
    if not gen % 100:
        live_labels = np.concatenate([x_alleles, y_alleles])
        DRIVES = DRIVES.loc[live_labels, live_labels]     
           
    # plotting data
    for x_allele, x_freq in zip(x_alleles, x_freqs):
        x_data[x_allele].append(gen)
        y_data[x_allele].append(x_freq)
    for y_allele, y_freq in zip(y_alleles, y_freqs):
        x_data[y_allele].append(gen)
        y_data[y_allele].append(y_freq)
    y_data['male_freq'].append(population.male_freq())
    x_data['male_freq'].append(gen)
    y_data['nr_x_alleles'].append(len(x_alleles))
    x_data['nr_x_alleles'].append(gen)
    y_data['nr_y_alleles'].append(len(y_alleles))
    x_data['nr_y_alleles'].append(gen)
    y_data['pop_size'].append(pop_size)
    x_data['pop_size'].append(gen)  


%matplotlib inline
plt.matshow(np.array(DRIVES)) ;

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5))
for i in range(0, 5000, 2):
    if not x_data[i]:
        break
    ax1.plot(x_data[i], y_data[i])
for i in range(1, 5000, 2):
    if not x_data[i]:
        break    
    ax2.plot(x_data[i], y_data[i])    

# if there is a lot of low freq alleles, is it then the high pi/higg ils regions that tend to sweep?

safas

In [ ]:
import plotly.graph_objects as go

frames = []
for g in range(0, 1000, 10):
    scatters = []
    for i in range(1000):
        scatters.append(go.Scatter(x=x_data[i][:g], y=y_data[i][:g]))
    frames.append(go.Frame(data=scatters))
        
fig = go.Figure(
    data=[go.Scatter(x=[0], y=[0]) for _ in range(1000)],
    layout=go.Layout(
        xaxis=dict(range=[0, 1000], autorange=False),
        yaxis=dict(range=[0, 1], autorange=False),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 50, 
                                                 "redraw": False},
                                                 "fromcurrent": True, 
                                                 "transition": {"duration": 10}}])])]

                                            
    ),
    frames=frames
)

fig.show()

In [ ]:
import plotly.graph_objects as go

xvals = np.linspace(1, 10, 100)
yvals = np.sin(xvals)

frames = []
for i in range(len(xvals)):
    frames.append(go.Frame(data=[go.Scatter(x=xvals[:i], y=yvals[:i]),
                                 go.Scatter(x=xvals[:i], y=[p-0.2 for p in yvals[:i]])]))

fig = go.Figure(
    data=[go.Scatter(x=[0, 1], y=[0, 1]), go.Scatter(x=[0, 1], y=[0, 1])],
    layout=go.Layout(
        xaxis=dict(range=[0, 10], autorange=False),
        yaxis=dict(range=[-1, 1], autorange=False),
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None, {"frame": {"duration": 50, 
                                                 "redraw": False},
                                                 "fromcurrent": True, 
                                                 "transition": {"duration": 0}}])])]

                                            
    ),
    frames=frames
)

fig.show()

In [ ]:
import plotly.express as px

df = px.data.gapminder()

fig = px.bar(df, x="continent", y="pop", color="continent",
  animation_frame="year", animation_group="country", range_y=[0,4000000000])
fig.show()